In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  4398k      0  0:00:18  0:00:17  0:00:01 5510k2M   69 55.4M    0     0  4094k      0  0:00:20  0:00:13  0:00:07 5411k    0  4444k      0  0:00:18  0:00:18 --:--:-- 5495k


In [2]:
!tar -xf aclImdb_v1.tar.gz

In [ ]:
from dataset import IMDBDataset
from utils import create_dataloaders
from transformer import Transformer

In [2]:
train_dir = "aclImdb/train"
train_dataloader, vocab = create_dataloaders(train_dir)

In [3]:
embed_dim = 256
num_heads = 2
dense_dim = 32

In [4]:
for batch in train_dataloader:
    text, label = batch
    encoder = Transformer(embed_dim, dense_dim, num_heads, 20000)
    output = encoder(text)
    print(output.shape)
    print(output)
    break


torch.Size([32, 600, 256])
tensor([[[-1.5797e+00, -5.8640e-01,  1.7663e+00,  ...,  1.0065e+00,
          -2.2506e-02,  8.2739e-01],
         [ 1.4732e-02, -5.6357e-01,  8.8516e-01,  ...,  6.6045e-01,
          -4.7447e-02, -1.3817e+00],
         [ 1.2706e+00, -7.1946e-01,  3.0590e-01,  ...,  7.6354e-01,
           2.9036e-01,  1.2190e+00],
         ...,
         [-4.7433e-01, -2.3487e+00,  9.1105e-01,  ..., -4.2399e-02,
           2.2378e-01,  1.2735e+00],
         [-4.7433e-01, -2.3487e+00,  9.1105e-01,  ..., -4.2399e-02,
           2.2378e-01,  1.2735e+00],
         [-4.7433e-01, -2.3487e+00,  9.1105e-01,  ..., -4.2399e-02,
           2.2378e-01,  1.2735e+00]],

        [[-5.6886e-01,  5.8067e-01, -6.9314e-01,  ...,  3.0544e-01,
           1.0172e+00,  2.8381e-03],
         [-6.3661e-01, -6.0064e-01,  1.2659e+00,  ..., -6.5288e-01,
          -2.8494e-01,  4.6201e-01],
         [ 1.0219e+00, -1.0889e+00, -2.7343e+00,  ..., -6.6192e-02,
           1.3628e+00,  6.4551e-01],
         ...